# 인스타 크롤링 데이터 전처리

+ 각 데이터 양 개수 확인하기 
    - **삼성   : 3243개 >>> 2459개**
    - **애플   : 5158개 >>> 2705개**
    - **샤오미 :**
    
+ 기업별 데이터 병합 후 한글을 제외한 키워드 삭제
+ 광고/홍보 글로 의심되는 문장 삭제


In [2]:
import numpy as np
import pandas as pd
import re

## processing 함수

+ processing()
+ clean_str()

In [3]:
def processing(data):
    # 데이터 중복 제거
    data["context"] = data["context"].drop_duplicates()
    data.dropna(inplace = True)
    data.reset_index(inplace = True, drop = True)
    print(data.info())

    # 데이터 필터링
    ban = ["개통", "팔로우", "좋반", "케이스", "프로필"]
    ban_idx = []
    try:
        for keyword in ban:
            for i in range(len(data)):
                if data["context"][i].find(keyword) != -1:
                    data = data.drop(index=i)
                    ban_idx.append(i)

                elif data["context"][i].find(keyword) == -1 :
                    pass   
            data.reset_index(inplace = True, drop = True)
            
    except Exception as ex :
        print(ex, "오류발생")
        pass
    

    return data

# 해당 행을 바로 삭제 or 인덱스 번호를 저장 후 삭제

In [4]:
# 한글만 살리기
def clean_str(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')    #한글만 남겨놓고 나머지 삭제
    s = hangul.sub(r'',str(s))

    cp = re.compile("["u"\U00010000-\U0010FFFF""]+", flags=re.UNICODE)  #BMP characters 이외 이모티콘삭제
    s=cp.sub(r'',s)

    s = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', s) #특수기호 삭제
 
    return s.strip()

### 삼성 checkpoints

In [4]:
##### 데이터 읽기
g_gear = pd.read_csv("갤럭시기어인스타.csv", header = None) #443개 
g_book = pd.read_csv("갤럭시북인스타.csv", header = None) # 639개
g = pd.read_csv("갤럭시인스타.csv", header = None) # 495개
g_tab = pd.read_csv("갤탭인스타.csv", header = None) # 469개
g_buzlive = pd.read_csv("버즈라이브인스타.csv", header = None) # 197개
g_buzplus = pd.read_csv("버즈플러스인스타.csv", header = None) # 1000개

In [5]:
##### 데이터 병합
s_df = pd.concat([g_gear, g_book, g, g_tab, g_buzplus, g_buzlive], ignore_index = True)
s_df.index

RangeIndex(start=0, stop=3243, step=1)

In [6]:
##### 한 게시물씩 리스트로 저장
samsung_data = []
for i in range(3243):
    samsung_data.append(s_df[1][i])

In [7]:
##### clean_str 함수 적용
s_list = []
for i in range(3243):
    s_list.append(clean_str(s_df[1][i]))
    samsung = pd.DataFrame({"context" : s_list})
samsung

,context
0,많이 기다리셨죠 나머지 품절되었던 상품들도 재입고되었으니 확인해보세요 유세라...
1,소통하고 지내고싶어요
2,셀프선물 세뱃돈플렉스 집에만 있기 너어무 지루하고 따분한 요즘셀프선물로 분위기전환하...
3,인디고오렌지 컬러조합 완벽한 가죽스트랩 유세라샵 갤럭시워치 갤럭시워치 갤럭시워치액티...
4,첫 조카가 생겼을 때 넘 예뻐서 물고 빨고 했는데벌써 그 아기가 커서 오늘 나한테 ...
...,...
3238,버즈라이브버즈프로야채친구들원구매시 프로필 링크클릭프로필링크 속의 일반 케이스 창고 ...
3239,버즈버즈플러스키링다이노랜드원구매시 프로필 링크클릭프로필링크 속의 일반 케이스 창고 ...
3240,버즈라이브 하드 케이스 커스텀으로 주문제작 해주신 고객님의 후기 사진입니다퀄리티가 ...
3241,하드톡앙보카도원구매시 프로필 링크클릭프로필링크 속의 일반 케이스 창고 에서 구매가능...


In [8]:
##### 병합한 데이터 출력
samsung.to_csv("samsung.csv")

In [9]:
##### 데이터 확인
samsung = pd.read_csv("samsung.csv", index_col = 0)
samsung.head()

,context
0,많이 기다리셨죠 나머지 품절되었던 상품들도 재입고되었으니 확인해보세요 유세라...
1,소통하고 지내고싶어요
2,셀프선물 세뱃돈플렉스 집에만 있기 너어무 지루하고 따분한 요즘셀프선물로 분위기전환하...
3,인디고오렌지 컬러조합 완벽한 가죽스트랩 유세라샵 갤럭시워치 갤럭시워치 갤럭시워치액티...
4,첫 조카가 생겼을 때 넘 예뻐서 물고 빨고 했는데벌써 그 아기가 커서 오늘 나한테 ...


In [10]:
##### processing 함수 적용
samsung_test = samsung
samsung_final = processing(samsung_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2549 entries, 0 to 2548
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   context  2549 non-null   object
dtypes: object(1)
memory usage: 20.0+ KB
None


In [11]:
samsung_final.to_csv("samsung_final.csv")

### 애플

In [55]:
##### 데이터 읽기
iphone = pd.read_csv("아이폰인스타.csv", header = None) #443개 
watch = pd.read_csv("애플워치인스타.csv", header = None) # 639개
airpod = pd.read_csv("에어팟인스타.csv", header = None) # 495개
macbook = pd.read_csv("맥북인스타.csv", header = None) # 469개
ipad = pd.read_csv("아이패드인스타.csv", header = None) # 197개
m1 = pd.read_csv("m1인스타.csv", header = None) # 1000개

In [56]:
##### 데이터 병합
a_df = pd.concat([iphone, watch, airpod, macbook, ipad, m1], ignore_index = True)
a_df.index

RangeIndex(start=0, stop=5158, step=1)

In [57]:
##### 한 게시물씩 리스트로 저장
apple_data = []
for i in range(len(a_df)):
    apple_data.append(a_df[1][i])

In [58]:
##### clean_str 함수 적용
a_list = []
for i in range(len(a_df)):
    a_list.append(clean_str(a_df[1][i]))
    apple = pd.DataFrame({"context" : a_list})
apple

,context
0,미우미우임메시와 리오넬메시 같이 있는 사진편집 올려요 ㅎ ㅎ 우리 웅님이 제일 좋아...
1,카페선일토요일 열려 있습니다무더위에도 찾아 주심에 감사합니다우유 도넛 과 짭쪼롭라...
2,좋은아침 예쁘네요 가짜인것처럼 진짜같은 꽃사진이네요 예뻐요 굿모닝입니다 토요일...
3,예술의전당 가면 꼭 가야하는 곳노을 지기전이 그렇게 이쁘더라구요요즘은 시 분 쯤인 ...
4,집앞 마트 가는데 땀범벅 밖으로 나가려고 엘리베이터 기다리는데 이미 땀이 송글 송...
...,...
5153,
5154,빠르게 달려도 바람이 많이 불어도 이어폰이 없어도 문제 없습니다 이마 위에 설치된 ...
5155,
5156,


In [64]:
apple['context'].replace('', np.nan, inplace=True)
apple.dropna(inplace = True)
apple

,context
0,미우미우임메시와 리오넬메시 같이 있는 사진편집 올려요 ㅎ ㅎ 우리 웅님이 제일 좋아...
1,카페선일토요일 열려 있습니다무더위에도 찾아 주심에 감사합니다우유 도넛 과 짭쪼롭라...
2,좋은아침 예쁘네요 가짜인것처럼 진짜같은 꽃사진이네요 예뻐요 굿모닝입니다 토요일...
3,예술의전당 가면 꼭 가야하는 곳노을 지기전이 그렇게 이쁘더라구요요즘은 시 분 쯤인 ...
4,집앞 마트 가는데 땀범벅 밖으로 나가려고 엘리베이터 기다리는데 이미 땀이 송글 송...
...,...
5102,이렇게 앱등이가 되는걸까요 지난 년동안 나름 충성스러운 삼성 유저였는데 재작년부터 ...
5108,미래의 내가 나한테 사준 것찐행복 진짜 찐이다 맥북케이스
5126,마이리얼스킨의슬기로운 집콕생활 집콕 챌린지요즘 새로운 일상으로 자리잡은 사회적거리두...
5136,거 반지가 너무 눈에 띄는거 아니오


In [16]:
##### 병합한 데이터 출력
apple.to_csv("apple.csv")

In [66]:
##### 데이터 확인
apple = pd.read_csv("apple.csv", index_col = 0)
apple.head()

,context
0,미우미우임메시와 리오넬메시 같이 있는 사진편집 올려요 ㅎ ㅎ 우리 웅님이 제일 좋아...
1,카페선일토요일 열려 있습니다무더위에도 찾아 주심에 감사합니다우유 도넛 과 짭쪼롭라...
2,좋은아침 예쁘네요 가짜인것처럼 진짜같은 꽃사진이네요 예뻐요 굿모닝입니다 토요일...
3,예술의전당 가면 꼭 가야하는 곳노을 지기전이 그렇게 이쁘더라구요요즘은 시 분 쯤인 ...
4,집앞 마트 가는데 땀범벅 밖으로 나가려고 엘리베이터 기다리는데 이미 땀이 송글 송...


In [67]:
##### processing 함수 적용
apple_test = apple
apple_final = processing(apple_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   context  4177 non-null   object
dtypes: object(1)
memory usage: 32.8+ KB
None


In [88]:
apple_final.to_csv("apple_final.csv")

### 샤오미

In [6]:
##### 데이터 읽기
miband = pd.read_csv("미밴드인스타.csv", header = None) 
hongmi = pd.read_csv("홍미노트인스타.csv", header = None) 
# watch = pd.read_csv("샤오미워치인스타.csv", header = None) 
amaze = pd.read_csv("어메이즈핏인스타.csv", header = None)
phone = pd.read_csv("샤오미폰인스타.csv", header = None)
miphone = pd.read_csv("샤오미휴대폰인스타.csv", header = None) 

In [9]:
##### 데이터 병합
m_df = pd.concat([miband, hongmi, amaze, phone, miphone], ignore_index = True)
m_df.index

RangeIndex(start=0, stop=948, step=1)

In [10]:
##### 한 게시물씩 리스트로 저장
mi_data = []
for i in range(len(m_df)):
    mi_data.append(m_df[1][i])

In [11]:
##### clean_str 함수 적용
m_list = []
for i in range(len(m_df)):
    m_list.append(clean_str(m_df[1][i]))
    mi = pd.DataFrame({"context" : m_list})
mi

,context
0,이거 써보신 분 계세요 이번에 구매했는데 넘 좋아서요 샤오미에서 나온 미밴드 인...
1,집에 도착해서보니 나의 미밴드 걸음수 줌바 에서 얼마나 깨작깨작 스텝 밟았으면 하기...
2,두 번 운동한 날은 스트레칭 쉬고 싶은데 운동 쉬고 싶은데 박하을 다이어트땜같이한다...
3,ㅎㅎ ㅎㅎㅎ ...
4,데이오프에 베트남으로 여행다녀옴 머리부터발끝까지 생일선물 ㅋㅋ효뜨 데이오프 다음엔...
...,...
943,블랙박스공기청정기 보조배터리샤오미휴대폰 진열제품구경공짜판플렛공짜남양주진접아이나비공식...
944,내꺼아님 탐나넹샤오미휴대폰 이거이거 그럴싸하쥬 완젼 탐났던아이 지문인식도됨좀더 꼼...
945,샤오미 미맥스 직구가격대비 가성비 좋네 ㅋㅋ휴대폰 가성비갑 샤오미미맥스 샤오미 데일...
946,샤오미휴대폰 카메라 나이성별이 인식댄다 ㅋ신기방기ㅋ 베이징 여행 좋은사람과 샤오미...


In [12]:
mi['context'].replace('', np.nan, inplace=True)
mi.dropna(inplace = True)
mi

,context
0,이거 써보신 분 계세요 이번에 구매했는데 넘 좋아서요 샤오미에서 나온 미밴드 인...
1,집에 도착해서보니 나의 미밴드 걸음수 줌바 에서 얼마나 깨작깨작 스텝 밟았으면 하기...
2,두 번 운동한 날은 스트레칭 쉬고 싶은데 운동 쉬고 싶은데 박하을 다이어트땜같이한다...
3,ㅎㅎ ㅎㅎㅎ ...
4,데이오프에 베트남으로 여행다녀옴 머리부터발끝까지 생일선물 ㅋㅋ효뜨 데이오프 다음엔...
...,...
943,블랙박스공기청정기 보조배터리샤오미휴대폰 진열제품구경공짜판플렛공짜남양주진접아이나비공식...
944,내꺼아님 탐나넹샤오미휴대폰 이거이거 그럴싸하쥬 완젼 탐났던아이 지문인식도됨좀더 꼼...
945,샤오미 미맥스 직구가격대비 가성비 좋네 ㅋㅋ휴대폰 가성비갑 샤오미미맥스 샤오미 데일...
946,샤오미휴대폰 카메라 나이성별이 인식댄다 ㅋ신기방기ㅋ 베이징 여행 좋은사람과 샤오미...


In [13]:
##### 병합한 데이터 출력
mi.to_csv("mi.csv")

In [14]:
##### 데이터 확인
mi = pd.read_csv("mi.csv", index_col = 0)
mi.head()

,context
0,이거 써보신 분 계세요 이번에 구매했는데 넘 좋아서요 샤오미에서 나온 미밴드 인...
1,집에 도착해서보니 나의 미밴드 걸음수 줌바 에서 얼마나 깨작깨작 스텝 밟았으면 하기...
2,두 번 운동한 날은 스트레칭 쉬고 싶은데 운동 쉬고 싶은데 박하을 다이어트땜같이한다...
3,ㅎㅎ ㅎㅎㅎ ...
4,데이오프에 베트남으로 여행다녀옴 머리부터발끝까지 생일선물 ㅋㅋ효뜨 데이오프 다음엔...


In [15]:
##### processing 함수 적용
mi_test = mi
mi_final = processing(mi)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   context  876 non-null    object
dtypes: object(1)
memory usage: 7.0+ KB
None


In [16]:
mi_final.to_csv("mi_final.csv")